In [7]:
import pandas as pd
import numpy as np
import sys,os
this_dir = os.path.abspath('.')
parent_dir = os.path.abspath('..')
if this_dir not in sys.path:
    sys.path.append(parent_dir)
    sys.path.append(this_dir)
import db_info#@UnresolvedImport
import typing
import plotly_utilities as pu
import datetime
from IPython import display


In [2]:
opttab = 'sec_schema.options_table'
futtab = 'sec_schema.underlying_table'

In [4]:
sqld = SqlDownloader(config_name='local')

  sec_db


In [3]:
StrList = typing.List[str]
class SqlDownloader():
    def __init__(self,config_name:str='local'):
        self.pga = db_info.get_db_info(config_name=config_name)
    
    def get_sql(
        self,
        columns=None,
        table=futtab,
        where_clauses=None,
        groupby_columns=None,
        orderby_columns=None,
        limit=None
    ):
        cols = '*' if columns is None else ','.join(columns)
        sql = f"select {cols} from {table}"
        
        if where_clauses is not None:
            wc_string = f'where {where_clauses[0]}'
            if len(where_clauses)>1:
                wc_string =  wc_string + " and " + ' and '.join(where_clauses[1:])
            wc_string = wc_string.replace('%','%%')
            sql = sql + " " + f"{wc_string}"
        if groupby_columns is not None:
            sql += f" group by {','.join(groupby_columns)}"
        if orderby_columns is not None:
            sql += f" order by {','.join(orderby_columns)}"
        if limit is not None:
            sql += f" limit {int(str(limit))}"
        df = self.pga.get_sql(sql)
        return df
        
    def get_options(self,symbol:str,yyyymmdd_beg:int=None,yyyymmdd_end:int=None):
        sql = f"select * from {opttab} where symbol='{symbol}'"
        if yyyymmdd_beg is not None:
            sql += f' and settle_date>={yyyymmdd_beg}'
        if yyyymmdd_end is not None:
            sql += f' and settle_date<={yyyymmdd_end}'
        df =  self.pga.get_sql(sql)
        if df is not None and len(df)>2:
            df = df.sort_values(['pc','settle_date','strike'])
        return df

    def get_futures(self,symbol_list:StrList,yyyymmdd_beg:int=None,yyyymmdd_end:int=None):
        if type(symbol_list) is list or type(symbol_list) is np.ndarray:
            sym_string = ",".join([f"'{s}'" for s in symbol_list]) 
        else:
            sym_string = f"'{symbol_list}'"
        sql = f"select * from {futtab} where symbol in ({sym_string})"
        if yyyymmdd_beg is not None:
            sql += f' and settle_date>={yyyymmdd_beg}'
        if yyyymmdd_end is not None:
            sql += f' and settle_date<={yyyymmdd_end}'
        df =  self.pga.get_sql(sql)
        if df is not None and len(df)>2:
            df = df.sort_values(['symbol','settle_date'])
        return df


In [43]:
def get_spread(c1,c2):
    df_c1 = sqld.get_futures(c1)[['settle_date','close']]
    df_c1 = df_c1.rename(columns={'close':c1})
    df_c2 = sqld.get_futures(c2)[['settle_date','close']]
    df_c2 = df_c2.rename(columns={'close':c2})
    df_both = df_c1.merge(df_c2,how='inner',on='settle_date')
    df_both[f"{c1}_{c2}"] = df_both[c1] - df_both[c2]
    return df_both

def get_spread_stats(c1,c2,beg_yyyymmdd=None,end_yyyymmdd=None):
    bd = beg_yyyymmdd
    if bd is None:
        # get year of c1
        bd = (int(c1[-2:])+2000)*100*100 + 101
    ed = end_yyyymmdd
    if ed is None:
        ed = (int(c2[-2:])+2000)*100*100 + 1231
    df_spr = get_spread(c1,c2)
    df_spr = df_spr[(df_spr.settle_date>=bd) & (df_spr.settle_date<=ed)]
    r = {}
    r['min'] = df_spr[f"{c1}_{c2}"].min()
    r['max'] = df_spr[f"{c1}_{c2}"].max()
    min_date = df_spr[df_spr[f"{c1}_{c2}"]<=r['min']].iloc[0].settle_date
    r['min_date'] = min_date
    max_date = df_spr[df_spr[f"{c1}_{c2}"]>=r['max']].iloc[0].settle_date
    r['max_date'] = max_date
    r['last_value'] = df_spr.iloc[-1][f"{c1}_{c2}"]
    return r

In [44]:
get_spread(c1,c2)

,settle_date,NGH21,NGN21,NGH21_NGN21
0,20110801,7.364,7.089,0.275
1,20110802,7.272,6.999,0.273
2,20110803,7.223,6.950,0.273
3,20110804,7.108,6.856,0.252
4,20110805,7.045,6.803,0.242
...,...,...,...,...
2379,20210111,2.703,2.842,-0.139
2380,20210112,2.707,2.841,-0.134
2381,20210113,2.689,2.843,-0.154
2382,20210114,2.630,2.802,-0.172


In [45]:
c1 = 'NGH21'
c2 = 'NGN21'
spr = []
min_spr = []
min_spr_date = []
max_spr = []
max_spr_date = []
last_value = []

for yy in range(13,22):
    yyyy = 2000 + yy
    bd = (yyyy-1)*100*100+101
    c1 = 'NGH' + str(yy)
    c2 = 'NGN' + str(yy)
    r = get_spread_stats(c1,c2,beg_yyyymmdd=bd)
    spr.append(f"{c1}_{c2}")
    min_spr.append(r['min'])
    min_spr_date.append(r['min_date'])
    max_spr.append(r['max'])
    max_spr_date.append(r['max_date'])
    last_value.append(r['last_value'])
df_spr_stats = pd.DataFrame(
    {'symbol':spr,'min_spr':min_spr,'min_spr_date':min_spr_date,
     'max_spr':max_spr,'max_spr_date':max_spr_date,'last_value':last_value}
)    
df_spr_stats

,symbol,min_spr,min_spr_date,max_spr,max_spr_date,last_value
0,NGH13_NGN13,-0.259,20130208.0,-0.026,20121019.0,-0.178
1,NGH14_NGN14,-0.101,20131105.0,1.362,20140219.0,0.336
2,NGH15_NGN15,-0.190,20150209.0,0.814,20140221.0,-0.080
3,NGH16_NGN16,-0.341,20151218.0,0.162,20150515.0,-0.308
4,NGH17_NGN17,-0.378,20170224.0,0.295,20160628.0,-0.378
5,NGH18_NGN18,-0.157,20180216.0,0.680,20170406.0,-0.144
6,NGH19_NGN19,-0.179,20190213.0,1.702,20181114.0,-0.029
7,NGH20_NGN20,-0.252,20200121.0,0.395,20190301.0,-0.199
8,NGH21_NGN21,-0.189,20201228.0,0.376,20200924.0,-0.160


In [52]:
pu.iplt(
    pu.plotly_plot(
        df_spr_stats[['symbol','min_spr','max_spr','last_value']],
        x_column='symbol',
        bar_plot=True,
        y_left_label="NGH-NGN spread High and Low",
        plot_title="Minimum and Maximum Spread Values for NGH/NGN since 2013<br>(negative values denote NGN over NGH)"
    )
)

In [4]:
#!jupyter nbconvert secdb_example.ipynb --to python